# Model testing

Now that all the metadata has been studied some testing with models from diferent sources and original code will be undergone in order to define some models to send to the boada environment.

We will start slow with simple pre-trained models extracted directly from the pytorch environment and build from those up with the modifications that we want to study in the experimentation. Mainly the models tested will be ResNet (in some of it's variants) and some DensNet variants at first with the option of further models if everything goes to plan.

Another objective would be to try to test the difference between transfer learning and full training from scratch, considering some other more sophisticated learning methods like one-shot if there is time.

Finally some degree of localization will be accquired with the advancements of the paper from Selvaraju et al. of Grad-CAM.  Code will be recicled with the original paper code with the necessary modifications in order to apply it to the selected models of the project.


## Preparing the split between test, train and validation

In [1]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

# Get the labels and read the original metadata
labels = ['Atelectasis',
          'Cardiomegaly',
          'Consolidation',
          'Edema',
          'Effusion',
          'Emphysema',
          'Fibrosis',
          'Hernia',
          'Infiltration',
          'Mass',
          'Nodule',
          'Pleural_Thickening',
          'Pneumonia',
          'Pneumothorax']

In [2]:
metadata = pd.read_csv('Data_Entry_2017_v2020.csv', delimiter=',')

# Encode the labels with multi-label friendly encoding
for label in labels:
    metadata[label] = metadata['Finding Labels'].apply(lambda x: 1 if label in x else 0)

metadata_positive = metadata[metadata['Finding Labels'] != 'No Finding']

metadata = metadata.drop(columns=['Finding Labels', 'Follow-up #','Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'])

# Get the test train and val splits according to the patient ID so no patients end up split between groups
gss_test = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_val_idx, test_idx = next(gss_test.split(metadata, groups=metadata['Patient ID']))

train_val_metadata = metadata.iloc[train_val_idx]
test_metadata = metadata.iloc[test_idx]

gss_train_val = GroupShuffleSplit(test_size=0.125, n_splits=1, random_state=42)
train_idx, val_idx = next(gss_train_val.split(train_val_metadata, groups=train_val_metadata['Patient ID']))

train_metadata = train_val_metadata.iloc[train_idx]
val_metadata = train_val_metadata.iloc[val_idx]


# Drop the column of patient ID
train_metadata = train_metadata.drop(columns=['Patient ID'])
val_metadata = val_metadata.drop(columns=['Patient ID'])
test_metadata = test_metadata.drop(columns=['Patient ID'])

#Write all the new metadata as csv to load easier
train_metadata.to_csv('./labels/train_metadata.csv', index=False)
val_metadata.to_csv('./labels/val_metadata.csv', index=False)
test_metadata.to_csv('./labels/test_metadata.csv', index=False)

In [3]:
metadata_positive.reset_index(drop=True, inplace=True)

metadata_positive = metadata_positive.drop(columns=['Finding Labels', 'Follow-up #','Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width','Height]', 'OriginalImagePixelSpacing[x', 'y]'])

train_val_idx, test_idx = next(gss_test.split(metadata_positive, groups=metadata_positive['Patient ID']))

train_val_metadata_p = metadata_positive.iloc[train_val_idx]
test_metadata_p = metadata_positive.iloc[test_idx]

train_idx, val_idx = next(gss_train_val.split(train_val_metadata_p, groups=train_val_metadata_p['Patient ID']))

train_metadata_p = train_val_metadata_p.iloc[train_idx]
val_metadata_p = train_val_metadata_p.iloc[val_idx]


# Drop the column of patient ID
train_metadata_p = train_metadata_p.drop(columns=['Patient ID'])
val_metadata_p = val_metadata_p.drop(columns=['Patient ID'])
test_metadata_p = test_metadata_p.drop(columns=['Patient ID'])


#Write all the new metadata as csv to load easier
train_metadata_p.to_csv('./labels/train_metadata_positive.csv', index=False)
val_metadata_p.to_csv('./labels/val_metadata_positive.csv', index=False)
test_metadata_p.to_csv('./labels/test_metadata_positive.csv', index=False)

In [4]:
print(f'in terms of positives train: {len(train_metadata_p)}, val: {len(val_metadata_p)}, test: {len(test_metadata_p)} and total: {len(train_metadata_p) + len(test_metadata_p) + len(val_metadata_p) }')

print(f'in terms of all train: {len(train_metadata)}, val: {len(val_metadata)}, test: {len(test_metadata)} and total: {len(train_metadata) + len(test_metadata) + len(val_metadata) }')

in terms of positives train: 36350, val: 5116, test: 10293 and total: 51759
in terms of all train: 78873, val: 10953, test: 22294 and total: 112120



## Dataset definition and load (dataset.py)

We define the dataset and load from the data entries

In [5]:
import torch
from torch.utils.data import Dataset
import os
import numpy as np
import pandas as pd
from PIL import Image

class ChestXRay(Dataset):
    def __init__(self, df_dir, image_dir, transform=None):
        """
        Args:
            df_dir: Path to the csv file with image names and labels.
            image_dir: Directory with all the images with the labels.
            transform: Optional transform to be applied on a sample.
        """
        self.data_frame = pd.read_csv(df_dir)
        self.image_dir = image_dir
        self.transform = transform

        self.size = len(self.data_frame)
        self.labels = np.array(self.data_frame.iloc[:, 1:])
        self.images = self.data_frame.iloc[:,0]

        #cambiar esta mier
        self.pred_label =  {'Atelectasis': 0,
                            'Cardiomegaly': 1,
                            'Consolidation': 2,
                            'Edema': 3,
                            'Effusion': 4,
                            'Emphysema': 5,
                            'Fibrosis': 6,
                            'Hernia': 7,
                            'Infiltration': 8,
                            'Mass': 9,
                            'Nodule': 10,
                            'Pleural_Thickening': 11,
                            'Pneumonia': 12,
                            'Pneumothorax': 13}
        
        self.classes = ['Atelectasis',
                        'Cardiomegaly',
                        'Consolidation',
                        'Edema',
                        'Effusion',
                        'Emphysema',
                        'Fibrosis',
                        'Hernia',
                        'Infiltration',
                        'Mass',
                        'Nodule',
                        'Pleural_Thickening',
                        'Pneumonia',
                        'Pneumothorax']
        

    def __getitem__(self, idx):
        # Get image path
        img_name = os.path.join(self.image_dir, self.images.iloc[idx])
        image = Image.open(img_name).convert("RGB")
        
        # Get labels
        labels = np.array(self.labels[idx])
        
        # Check for any object types (invalid data)
        if not np.issubdtype(labels.dtype, np.number):
            raise TypeError(f"Non-numeric label detected at index {idx}: {labels}")
        
        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        image = np.array(image)
        
        return {'image': image, 'labels': labels}
    
    def __len__(self):
        return self.size

## Various functions (utils.py)



In [55]:
import torchvision
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import os

normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])


def make_data_loaders(train_csv,val_csv,image_dir,batch_size,image_size):
    

    train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    normalize
    ])

    val_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    normalize
    ])

    train_dataset = ChestXRay(df_dir=train_csv, image_dir=image_dir, transform=train_transforms)
    val_dataset = ChestXRay(df_dir=val_csv, image_dir=image_dir, transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    dataloaders = {'train': train_loader, 'val': val_loader}

    return dataloaders, {'train':len(train_dataset),'val':len(train_dataset)}, train_dataset.classes

In [56]:
import torch
import os

def LoadModel(checkpoint_file, model, optimizer, epoch_inti, best_auc_ave, num_GPU):
    '''
    The loads the model, optimizer, current epoch, and current validation AUC from the checkpoint location provided.

    Parameters
    ----------
    checkpoint_file: (str) the location of the model in s/m
    model: PyTorch model
    optimizer: PyTorch optimizer
    epoch_inti: current epoch
    best_auc_ave: current best AUC
    num_GPU : (int) number of GPUs that are being used

    Returns
    -------
    Returns the model, optimizer, epoch_inti, best_auc_ave from the saved location.
    '''
    checkpoint = torch.load(checkpoint_file)
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_inti = checkpoint['epoch']
    best_auc_ave = checkpoint['best_va_acc']
    if num_GPU > 1:
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    return model, optimizer, epoch_inti, best_auc_ave


def SaveModel(epoch, model, optimizer, best_auc_ave, file_name, num_GPU):
    """
    Save the model parameters, optimizer, best_AUC

    Parameters
    ----------
    epoch : (int) current epoch
    model : Pytorch model to save
    optimizer : Pytorch optimzer to save
    best_auc_ave : (float) current best AUC
    file_name : (str) location where the model needed to be saved
    num_GPU : (int) number of GPUs that are being used

    """
    if num_GPU > 1:
        state = {
            'epoch': epoch,
            'model_state_dict': model.module.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_va_acc': best_auc_ave
        }
    else:
        state = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_va_acc': best_auc_ave
        }
    torch.save(state, file_name)
    pass

In [30]:
import torch.nn.functional as F
from torch.optim import SGD, Adadelta, Adagrad, Adam, RMSprop

def get_optimizer(params, optimizer, lr=1e-4, momentum=0.9, weight_decay=0.0):
    """
    Loads and returns the optimizer.
    """
    if optimizer == 'SGD':
        return SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer == 'Adadelta':
        return Adadelta(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'Adagrad':
        return Adagrad(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'Adam':
        return Adam(params, lr=lr, weight_decay=weight_decay)
    elif optimizer == 'RMSprop':
        return RMSprop(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise Exception('Unknown optimizer : {}'.format(optimizer))


def weighted_BCELoss(output, target, weights=None):
    '''
    The function computes the weighted Binary cross Entropy loss.
    Parameters
    ----------
    output :  Predicted value from model. (tensor NX8 for multi-label classifcation or Nx1 for binary classification.)
    target : (tensor) Ground truth label. (tensor NX8 for multi-label classifcation or Nx1 for binary classification.)
    weights : the weights (float tensor)

    Returns
    -------
    loss : the WBCE in float tensor
    '''
    output = output.clamp(min=1e-5, max=1 - 1e-5)
    target = target.float()
    if weights is not None:
        assert len(weights) == 2
        loss = -weights[0] * (target * torch.log(output)) - weights[1] * ((1 - target) * torch.log(1 - output))
    else:
        loss = -target * torch.log(output) - (1 - target) * torch.log(1 - output)
    return torch.sum(loss)

def lr_schedule(lr, lr_factor, epoch_now, lr_epochs):
    """
    Learning rate schedule with respect to epoch
    lr: float, initial learning rate
    lr_factor: float, decreasing factor every epoch_lr
    epoch_now: int, the current epoch
    lr_epochs: list of int, decreasing every epoch in lr_epochs
    return: lr, float, scheduled learning rate.
    """
    count = 0
    for epoch in lr_epochs:
        if epoch_now >= epoch:
            count += 1
            continue
        break
    return lr * np.power(lr_factor, count)


def get_loss(output, target, index, device, cfg):
    target = target[:, index].view(-1)
    if target.sum() == 0:
        loss = torch.tensor(0., requires_grad=True).to(device)
    else:
        weight = (target.size()[0] - target.sum()) / target.sum()
        loss = F.binary_cross_entropy_with_logits(
            output[:, index].view(-1), target.float(), pos_weight=weight)
    label = torch.sigmoid(output[:, index].view(-1)).ge(0.5).float()
    acc = (target == label).float().sum() / len(label)
    return (loss, acc)

## Training method and validation of training

Basic training method based on epochs and iterating through all the dataset

In [41]:
import torch
import copy
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from sklearn.metrics import roc_auc_score
import numpy as np
import time
from tqdm import tqdm
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_save_dir = './saves'
image_dir = './resized_images'
log_dir = './logs'
train_csv = './labels/train_metadata_positive.csv '
val_csv = './labels/val_metadata_positive.csv'
num_classes = 14
# Dataset mode would be nice to alternate through positive and non positive metadata

def train_model_1(model, model_name, optimizer, lr, batch_size, image_size, num_epochs, resume=False, weighted_loss=False):

    # Writer to write logs for the model
    writer = SummaryWriter(log_dir=os.path.join(log_dir, model_name))
    #writer.add_text('log', str(args), 0)

    # Get the optimizer
    params = model.parameters()
    optimizer = get_optimizer(params, optimizer, lr) #momentum, weight_decay are optionals to be added if necessary

    # Get the data loaders
    dataloaders, dataset_sizes, class_names = make_data_loaders(train_csv,val_csv,image_dir,batch_size,image_size)

    # Variables in order to store the state of training
    best_auc_ave = 0.0  # to check if best validation accuracy
    epoch_inti = 1  # epoch starts from here

    best_model_wts = model.state_dict()
    best_auc = []
    iter_num = 0

    # Prepare to save the model
    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)
    
    checkpoint_file = os.path.join(model_save_dir, str(model_name + '_' + "checkpoint.pth"))
    bestmodel_file = os.path.join(model_save_dir, str(model_name + '_' + "best_model.pth"))

    ''' Check for existing training results. If it existst, and the configuration
    is set to resume `resume==True`, resume from previous training. 
    If not, delete existing checkpoint.'''
    if os.path.exists(checkpoint_file):
        if resume:
            model, optimizer, epoch_inti, best_auc_ave = LoadModel(checkpoint_file, model, optimizer, epoch_inti,
                                                                   best_auc_ave)
            print("Checkpoint found! Resuming")
        else:
            pass

    print(model)
    since = time.time()

    for epoch in range(epoch_inti, num_epochs + 1):
        model.train()
        running_loss = 0.0

        print(f'Starting epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train','val']:
            # Only Validation in every 5 cycles
            if (phase == 'val') :
                model.train(False)
            elif phase == 'train':
                model.train(True)  # Set model to training mode


            running_loss = 0.0
            output_list = []
            label_list = []
            loss_list = []

            # Iterate over data.
            for idx, data in enumerate(tqdm(dataloaders[phase])):


                if iter_num > 100 and phase == 'train':
                    break
                images = data['image']
                labels = data['labels']
                images = images.to(device)
                labels = labels.to(device)

                mask = 1 * (labels >= 0)
                mask = mask.to(device)

                if phase == 'train':
                    torch.set_grad_enabled(True)
                else:
                    torch.set_grad_enabled(False)

                # calculate weight for loss
                P = 0
                N = 0
                for label in labels:
                    for v in label:
                        if int(v) == 1:
                            P += 1
                        else:
                            N += 1
                if P != 0 and N != 0:
                    BP = (P + N) / P
                    BN = (P + N) / N
                    weights = torch.tensor([BP, BN], dtype=torch.float).to(device)
                else:
                    weights = None

                # zero the parameter gradients
                optimizer.zero_grad()
                # Predicting output
                outputs = model(images)

                loss = weighted_BCELoss(outputs, labels, weights=weights)
                

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    iter_num += 1

                running_loss += loss.item()
                loss_list.append(loss.item())
                outputs = outputs.detach().to('cpu').numpy()
                labels = labels.detach().to('cpu').numpy()

                for i in range(outputs.shape[0]):
                    output_list.append(np.where(labels[i] >= 0, outputs[i], 0).tolist())
                    label_list.append(np.where(labels[i] >= 0, labels[i], 0).tolist())

                # Saving logs
                if idx % 100 == 0 and idx != 0:
                    if phase == 'train':
                        writer.add_scalar('loss/train_batch', loss.item() / outputs.shape[0], iter_num)
                        try:
                            auc = roc_auc_score(np.array(label_list[-100 * batch_size:]),
                                                np.array(output_list[-100 * batch_size:]))
                            writer.add_scalar('auc/train_batch', auc, iter_num)
                            print('\nAUC/Train', auc)
                            print('Batch Loss', sum(loss_list) / len(loss_list))
                        except:
                            pass

            epoch_loss = running_loss / dataset_sizes[phase]
            # Computing AUC
            try:
                epoch_auc_ave = roc_auc_score(np.array(label_list), np.array(output_list))
                epoch_auc = roc_auc_score(np.array(label_list), np.array(output_list), average=None)
            except ValueError:
                epoch_auc_ave = roc_auc_score(np.array(label_list)[:, :12], np.array(output_list)[:, :12]) * (12 / 13) + \
                                roc_auc_score(np.array(label_list)[:, 13], np.array(output_list)[:, 13]) * (1 / 13)
                epoch_auc = roc_auc_score(np.array(label_list)[:, :12], np.array(output_list)[:, :12], average=None)
                epoch_auc = np.append(epoch_auc, 0)
                epoch_auc = np.append(epoch_auc, roc_auc_score(np.array(label_list)[:, 13], np.array(output_list)[:, 13], average=None))

            except:
                epoch_auc_ave = 0
                epoch_auc = [0 for _ in range(len(class_names))]

            if phase == 'val':
                writer.add_scalar('loss/validation', epoch_loss, epoch)
                writer.add_scalar('auc/validation', epoch_auc_ave, epoch)
            if phase == 'train':
                writer.add_scalar('loss/train', epoch_loss, epoch)
                writer.add_scalar('auc/train', epoch_auc_ave, epoch)

            log_str = ''
            log_str += 'Loss: {:.4f} AUC: {:.4f}  \n\n'.format(
                epoch_loss, epoch_auc_ave)

            for i, c in enumerate(class_names):
                log_str += '{}: {:.4f}  \n'.format(c, epoch_auc[i])

            log_str += '\n'
            if phase == 'val':
                print("\n\nValidation Phase ")
            else:
                print("\n\nTraining Phase ")

            print(log_str)
            writer.add_text('log', log_str, iter_num)
            print("Best validation average AUC :", best_auc_ave)
            print("Average AUC of current epoch :", epoch_auc_ave)

            # save model with best validation AUC
            if phase == 'val' and epoch_auc_ave > best_auc_ave:
                best_auc = epoch_auc
                print("Rewriting model with AUROC :", round(best_auc_ave, 4), " by model with AUROC : ",
                      round(epoch_auc_ave, 4))
                best_auc_ave = epoch_auc_ave
                print('Model saved to %s' % bestmodel_file)
                print("Saving the best checkpoint")
                SaveModel(epoch, model, optimizer, best_auc_ave, bestmodel_file)

            if phase == 'train' and epoch % 1 == 0:
                SaveModel(epoch, model, optimizer, best_auc_ave, checkpoint_file)

        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val AUC: {:4f}'.format(best_auc_ave))
    print()
    try:
        for i, c in enumerate(class_names):
            print('{}: {:.4f} '.format(c, best_auc[i]))
    except:
        for i, c in enumerate(class_names):
            print('{}: {:.4f} '.format(c, epoch_auc[i]))


    # load best model weights to return
    model.load_state_dict(best_model_wts)

    return model

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.nn import DataParallel
from torchvision.models import ResNet18_Weights, ResNet50_Weights, DenseNet121_Weights

resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 14 - 1)

model = resnet18
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = DataParallel(model)
model.to(device)
train_model1(model, "resnet_18", "SGD", 1e-4, 16, 224, 20)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [6]:
import torch
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_model_wts = copy.deepcopy(model.state_dict())  # Keep track of best model weights
    best_val_loss = float('inf')  # Initialize best validation loss as infinity
    epochs_without_improvement = 0  # Counter for epochs without improvement
    
    # Define the learning rate scheduler (reduce LR on plateau)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        print(f'Starting epoch {epoch+1}/{num_epochs}')
        
        # Training loop
        for i, batch in enumerate(train_loader):
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            if i % 100 == 99:  # Print every 10 batches
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
                running_loss = 0.0
        
        # Validation at the end of each epoch
        val_loss = validate_model(model, val_loader, criterion)

        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss / len(train_loader):.4f}, Validation Loss: {val_loss:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss  # Update best validation loss
            best_model_wts = copy.deepcopy(model.state_dict())  # Save the best model weights
            epochs_without_improvement = 0  # Reset counter
        else:
            epochs_without_improvement += 1
            print(f'No improvement in validation loss for {epochs_without_improvement} epoch(s).')

        if epochs_without_improvement >= patience:
            print(f'Early stopping after {epochs_without_improvement} epochs without improvement.')
            break

        # Update the learning rate based on validation loss
        scheduler.step(val_loss)

    # Load best model weights before returning
    model.load_state_dict(best_model_wts)
    print('Training complete. Best Validation Loss:', best_val_loss)
    return model

def validate_model(model, val_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device).float()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    return val_loss / len(val_loader)  # Return average validation loss



## Test function

Test function that generates the testing values that are necessary for the evaluation of the dataset.

In [7]:
def test_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    threshold = 0.5  # Threshold for multi-label classification
    
    with torch.no_grad():
        for batch in test_loader:
            images = batch['image'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > threshold).float()

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

## Set paths and hyperparameters

In [8]:
# Hyperparameters
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# Paths
train_csv = './labels/train_metadata.csv '
train_image_dir = './resized_images'
val_csv = './labels/val_metadata.csv'
val_image_dir = './resized_images'
train_p_csv = './labels/train_metadata_positive.csv '
val_p_csv = './labels/val_metadata_positive.csv'

## Data loaders

In [10]:
from torch.utils.data import DataLoader

from torchvision import transforms

import os

normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    normalize
    ])

val_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    normalize
    ])

train_dataset = ChestXRay(df_dir=train_csv, image_dir=train_image_dir, transform=train_transforms)
val_dataset = ChestXRay(df_dir=val_csv, image_dir=val_image_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_dataset_p = ChestXRay(df_dir=train_p_csv, image_dir=train_image_dir, transform=train_transforms)
val_dataset_p = ChestXRay(df_dir=val_p_csv, image_dir=val_image_dir, transform=val_transforms)

train_loader_p = DataLoader(train_dataset_p, batch_size=batch_size, shuffle=True)
val_loader_p = DataLoader(val_dataset_p, batch_size=batch_size, shuffle=False)

## Models

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import ResNet18_Weights, ResNet50_Weights, DenseNet121_Weights

In [12]:
torch.cuda.is_available()

True

In [13]:
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, len(train_dataset_p.data_frame.columns) - 1)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=learning_rate)

# Train the model
train_model(resnet18, train_loader_p, val_loader_p, criterion, optimizer, num_epochs)

# Test the model
#test_model(resnet18, val_loader_p)

c:\Users\hugoa\anaconda3\envs\cuda_dev_1\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting epoch 1/10
Epoch [1/10], Step [100/2272], Loss: 0.3250
Epoch [1/10], Step [200/2272], Loss: 0.3081
Epoch [1/10], Step [300/2272], Loss: 0.3047
Epoch [1/10], Step [400/2272], Loss: 0.3101
Epoch [1/10], Step [500/2272], Loss: 0.3028
Epoch [1/10], Step [600/2272], Loss: 0.3017
Epoch [1/10], Step [700/2272], Loss: 0.3002
Epoch [1/10], Step [800/2272], Loss: 0.3017
Epoch [1/10], Step [900/2272], Loss: 0.3037
Epoch [1/10], Step [1000/2272], Loss: 0.3039
Epoch [1/10], Step [1100/2272], Loss: 0.3019
Epoch [1/10], Step [1200/2272], Loss: 0.3007
Epoch [1/10], Step [1300/2272], Loss: 0.3042
Epoch [1/10], Step [1400/2272], Loss: 0.2951
Epoch [1/10], Step [1500/2272], Loss: 0.2948
Epoch [1/10], Step [1600/2272], Loss: 0.3024
Epoch [1/10], Step [1700/2272], Loss: 0.2973
Epoch [1/10], Step [1800/2272], Loss: 0.2970
Epoch [1/10], Step [1900/2272], Loss: 0.3003
Epoch [1/10], Step [2000/2272], Loss: 0.2975
Epoch [1/10], Step [2100/2272], Loss: 0.2998
Epoch [1/10], Step [2200/2272], Loss: 0.2946

KeyboardInterrupt: 

In [33]:
save_path = "./model_checkpoint_resnet18_v1_10ep_p.pth"

store_model(resnet18, optimizer, num_epochs + 1, save_path)

Model checkpoint saved at ./model_checkpoint_resnet18_v1_10ep_p.pth


In [14]:
resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, len(train_dataset_p.data_frame.columns) - 1)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=learning_rate)

# Train the model
train_model(resnet50, train_loader_p, val_loader_p, criterion, optimizer, num_epochs)

# Test the model
#test_model(resnet50, val_loader_p)

c:\Users\hugoa\anaconda3\envs\cuda_dev_1\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting epoch 1/10
Epoch [1/10], Step [100/2272], Loss: 0.3198
Epoch [1/10], Step [200/2272], Loss: 0.2995
Epoch [1/10], Step [300/2272], Loss: 0.2932
Epoch [1/10], Step [400/2272], Loss: 0.2928
Epoch [1/10], Step [500/2272], Loss: 0.3001
Epoch [1/10], Step [600/2272], Loss: 0.2937
Epoch [1/10], Step [700/2272], Loss: 0.2936
Epoch [1/10], Step [800/2272], Loss: 0.2869
Epoch [1/10], Step [900/2272], Loss: 0.2833
Epoch [1/10], Step [1000/2272], Loss: 0.2815
Epoch [1/10], Step [1100/2272], Loss: 0.2860
Epoch [1/10], Step [1200/2272], Loss: 0.2854
Epoch [1/10], Step [1300/2272], Loss: 0.2805
Epoch [1/10], Step [1400/2272], Loss: 0.2832
Epoch [1/10], Step [1500/2272], Loss: 0.2819
Epoch [1/10], Step [1600/2272], Loss: 0.2763
Epoch [1/10], Step [1700/2272], Loss: 0.2836
Epoch [1/10], Step [1800/2272], Loss: 0.2824
Epoch [1/10], Step [1900/2272], Loss: 0.2768
Epoch [1/10], Step [2000/2272], Loss: 0.2827
Epoch [1/10], Step [2100/2272], Loss: 0.2790
Epoch [1/10], Step [2200/2272], Loss: 0.2798

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
save_path = "./model_checkpoint_resnet50_v1_10ep_p.pth"

store_model(resnet50, optimizer, num_epochs + 1, save_path)

Model checkpoint saved at ./model_checkpoint_resnet50_v1_10ep_p.pth


In [17]:
dense121 = models.densenet121(weights=DenseNet121_Weights.DEFAULT)
num_ftrs = dense121.classifier.in_features
dense121.classifier = nn.Linear(num_ftrs, len(train_dataset_p.data_frame.columns) - 1)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(dense121.parameters(), lr=learning_rate)

# Train the model
train_model(dense121, train_loader_p, val_loader_p, criterion, optimizer, num_epochs)

# Test the model
test_model(dense121, val_loader)

c:\Users\hugoa\anaconda3\envs\cuda_dev_1\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting epoch 1/10
Epoch [1/10], Step [100/2272], Loss: 0.3198
Epoch [1/10], Step [200/2272], Loss: 0.3023
Epoch [1/10], Step [300/2272], Loss: 0.2998
Epoch [1/10], Step [400/2272], Loss: 0.3036
Epoch [1/10], Step [500/2272], Loss: 0.2958
Epoch [1/10], Step [600/2272], Loss: 0.2985
Epoch [1/10], Step [700/2272], Loss: 0.3031
Epoch [1/10], Step [800/2272], Loss: 0.3031
Epoch [1/10], Step [900/2272], Loss: 0.3045
Epoch [1/10], Step [1000/2272], Loss: 0.2978
Epoch [1/10], Step [1100/2272], Loss: 0.2988
Epoch [1/10], Step [1200/2272], Loss: 0.3044
Epoch [1/10], Step [1300/2272], Loss: 0.3014
Epoch [1/10], Step [1400/2272], Loss: 0.3008
Epoch [1/10], Step [1500/2272], Loss: 0.2976
Epoch [1/10], Step [1600/2272], Loss: 0.2981
Epoch [1/10], Step [1700/2272], Loss: 0.2963
Epoch [1/10], Step [1800/2272], Loss: 0.3024
Epoch [1/10], Step [1900/2272], Loss: 0.2932
Epoch [1/10], Step [2000/2272], Loss: 0.2955
Epoch [1/10], Step [2100/2272], Loss: 0.2958
Epoch [1/10], Step [2200/2272], Loss: 0.2895

In [18]:
save_path = "./model_checkpoint_dense121_v1_10ep_p.pth"

store_model(dense121, optimizer, num_epochs + 1, save_path)

Model checkpoint saved at ./model_checkpoint_dense121_v1_10ep_p.pth
